In [1]:
input_image_name = "Image_Image"
input_browser_name = "LandmarkingScan"
skeleton_model_name = "SkeletonModel"
spine_volume_name = input_browser_name + "ReconstructionResults"

In [2]:
# Switch to red-only layout

layoutManager = slicer.app.layoutManager()
layoutManager.setLayout(slicer.vtkMRMLLayoutNode.SlicerLayoutOneUpRedSliceView)

In [3]:
# Move input image off of any transforms, and delete all other images

input_image_node = slicer.util.getFirstNodeByName(input_image_name, className="vtkMRMLScalarVolumeNode")
if input_image_node is None:
    raise Exception("Cannot find image: {}".format(input_image_name))

input_image_node.SetAndObserveTransformNodeID(None)

volume_nodes = slicer.mrmlScene.GetNodesByClass('vtkMRMLScalarVolumeNode')
volume_nodes.InitTraversal()
volume = volume_nodes.GetNextItemAsObject()
while volume is not None:
    volume_name = volume.GetName()
    if volume_name != input_image_name and volume_name != spine_volume_name:
        print("Delete {}".format(volume_name))
        slicer.mrmlScene.RemoveNode(volume)
    else:
        print("Keep   {}".format(volume_name))
    volume = volume_nodes.GetNextItemAsObject()


Delete Volume
Keep   Image_Image
Delete Image_Reference
Delete Segmented_Image
Keep   LandmarkingScanReconstructionResults
Delete LandmarkingScan_1ReconstructionResults
Delete LandmarkingScan_2ReconstructionResults
Delete LandmarkingScan_3ReconstructionResults
Delete Volume_1


In [4]:
# Show image in red view
redSliceWidget = slicer.app.layoutManager().sliceWidget('Red')
compNode = redSliceWidget.sliceLogic().GetSliceCompositeNode()
compNode.SetBackgroundVolumeID(input_image_node.GetID())
redSliceWidget.fitSliceToBackground()

redNode = slicer.mrmlScene.GetNodeByID('vtkMRMLSliceNodeRed')

resliceLogic = slicer.modules.volumereslicedriver.logic()
resliceLogic.SetDriverForSlice(input_image_node.GetID(), redNode)

# custom layout
customLayout = """
<layout type="horizontal" split="true">
    <item>
        <view class="vtkMRMLSliceNode" singletontag="Red">
            <property name="orientation" action="default">Axial</property>
            <property name="viewlabel" action="default">R</property>
            <property name="viewcolor" action="default">#F34A33</property>
        </view>
    </item>
    <item>
        <view class="vtkMRMLViewNode" singletontag="1">
            <property name="viewlabel" action="default">1</property>
        </view>
    </item>
</layout>
"""

customLayoutId = 501

layoutManager = slicer.app.layoutManager()
layoutManager.layoutLogic().GetLayoutNode().AddLayoutDescription(customLayoutId, customLayout)

customLayout = """
    <layout type="horizontal" split="true">
      <item>
       <view class="vtkMRMLSliceNode" singletontag="Red">
        <property name="orientation" action="default">Axial</property>
        <property name="viewlabel" action="default">R</property>
        <property name="viewcolor" action="default">#F34A33</property>
       </view>
      </item>
      <item>
        <layout type=\"horizontal\">
          <item>
            <view class="vtkMRMLViewNode" singletontag="1">
              <property name="viewlabel" action="default">1</property>
            </view>
          </item>
          <item>
            <view class="vtkMRMLViewNode" singletontag="2" type="secondary">
              <property name="viewlabel" action="default">2</property>
            </view>
          </item>
        </layout>
      </item>
    </layout>
    """

customLayoutId = 502

layoutManager.layoutLogic().GetLayoutNode().AddLayoutDescription(customLayoutId, customLayout)
layoutManager.setLayout(customLayoutId)

In [5]:
# Delete all sequence browsers that are not the input

sequence_browsers = slicer.mrmlScene.GetNodesByClass('vtkMRMLSequenceBrowserNode')
sequence_browsers.InitTraversal()
browser = sequence_browsers.GetNextItemAsObject()
while browser is not None:
    browser_name = browser.GetName()
    if browser_name != input_browser_name:
        print("Delete {}".format(browser_name))
        slicer.mrmlScene.RemoveNode(browser)
    else:
        print("Keep   {}".format(browser_name))
    browser = sequence_browsers.GetNextItemAsObject()

Delete RoadmapScan
Keep   LandmarkingScan
Delete LandmarkingScan_1
Delete LandmarkingScan_2
Delete LandmarkingScan_3


In [6]:
# Delete all sequences that don't belong to the input browser

input_browser_node = slicer.util.getFirstNodeByName(input_browser_name)
if input_browser_node is None:
    raise Exception("Cannot find {}".format(input_browser_name))

sequence_nodes_to_keep = vtk.vtkCollection()
input_browser_node.GetSynchronizedSequenceNodes(sequence_nodes_to_keep, True)

sequences = slicer.mrmlScene.GetNodesByClass('vtkMRMLSequenceNode')
sequences.InitTraversal()
seq = sequences.GetNextItemAsObject()
while seq is not None:
    if sequence_nodes_to_keep.IsItemPresent(seq):
        print("Keep   {}".format(seq.GetName()))
    else:
        print("Delete {}".format(seq.GetName()))
        slicer.mrmlScene.RemoveNode(seq)
    seq = sequences.GetNextItemAsObject()

Delete RoadmapScan-TransdToReference-Seq
Delete RoadmapScan-Image_Image-Seq
Delete RoadmapScan-ImageToTransd-Seq
Keep   LandmarkingScan-TransdToReference-Seq
Keep   LandmarkingScan-Image_Image-Seq
Keep   LandmarkingScan-ImageToTransd-Seq
Delete LandmarkingScan_1-TransdToReference-Seq
Delete LandmarkingScan_1-Image_Image-Seq
Delete LandmarkingScan_1-ImageToTransd-Seq
Delete LandmarkingScan_2-TransdToReference-Seq
Delete LandmarkingScan_2-Image_Image-Seq
Delete LandmarkingScan_2-ImageToTransd-Seq
Delete LandmarkingScan_3-TransdToReference-Seq
Delete LandmarkingScan_3-Image_Image-Seq
Delete LandmarkingScan_3-ImageToTransd-Seq


In [7]:
# Create segmentation

segmentation_node = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSegmentationNode", "Segmentation")
segmentation_node.CreateDefaultDisplayNodes()
segmentation_node.SetReferenceImageGeometryParameterFromVolumeNode(input_image_node)
segmentation_node.GetSegmentation().AddEmptySegment()

'Segment_1'

In [8]:
# Create output browser

output_browser = slicer.mrmlScene.AddNewNodeByClass("vtkMRMLSequenceBrowserNode", "SegmentationBrowser")
sequence_browser_logic = slicer.modules.sequences.logic()

segmentation_sequence = sequence_browser_logic.AddSynchronizedNode(None, None, output_browser)
segmentation_sequence.SetName("SegmentationSequence")
sequence_browser_logic.AddSynchronizedNode(segmentation_sequence, segmentation_node, output_browser)
output_browser.SetRecording(segmentation_sequence, True)
output_browser.SetSaveChanges(segmentation_sequence, False)

image_sequence = sequence_browser_logic.AddSynchronizedNode(None, None, output_browser)
image_sequence.SetName("ImageSequence")
sequence_browser_logic.AddSynchronizedNode(image_sequence, input_image_node, output_browser)
output_browser.SetRecording(image_sequence, True)
output_browser.SetSaveChanges(image_sequence, False)

output_browser.SetRecordMasterOnly(True)

In [9]:
# Show sequence browser toolbar, and select input browser in it

sb_toolbar = slicer.modules.sequences.toolBar()
sb_toolbar.setVisible(True)

input_browser_node = slicer.util.getFirstNodeByName(input_browser_name, className="vtkMRMLSequenceBrowserNode")
slicer.modules.sequences.setToolBarActiveBrowserNode(input_browser_node)

In [10]:
# place skeleton model in first 3d view
skeleton_volume = slicer.util.getFirstNodeByName(skeleton_model_name)
viewNode = layoutManager.threeDWidget(0).mrmlViewNode()
displayNode = skeleton_volume.GetDisplayNode()
displayNode.SetViewNodeIDs([viewNode.GetID()])

# place reconstructed volume in second 3d view
spine_volume = slicer.util.getFirstNodeByName(spine_volume_name)
viewNode = layoutManager.threeDWidget(1).mrmlViewNode()
displayNode = slicer.modules.volumerendering.logic().GetFirstVolumeRenderingDisplayNode(spine_volume)
displayNode.SetViewNodeIDs([viewNode.GetID()])
spine_volume.SetDisplayVisibility(1)